
# NumPy: Zero to Hero 🚀

Welcome! This notebook teaches **NumPy** from the ground up—fully self-paced and hands-on.
You will learn arrays, shapes, broadcasting, indexing, vectorization, linear algebra,
random generation, and performance tips.

**You will:**
- Understand arrays (`ndarray`), shapes, and dtypes
- Master indexing & slicing, boolean/fancy indexing
- Use broadcasting and axis-based reductions with confidence
- Reshape, stack, split, sort, search
- Handle missing values (`NaN`, `Inf`) and numerical stability basics
- Apply vectorization & ufuncs (universal functions)
- Perform core linear algebra (dot, matmul, solve, norms, eig/SVD overview)
- Generate random numbers with `numpy.random.Generator`
- Benchmark vectorized vs looped Python

Let's go! 💪


## 0) Setup

In [ ]:

import numpy as np

# For reproducibility in examples
rng = np.random.default_rng(42)
np.__version__



## 1) `ndarray` Basics

**What is an `ndarray`?**
- A **homogeneous** multidimensional array of fixed-size items.
- Fast and memory-efficient compared to Python lists.
- Supports **vectorized** operations (operate on whole arrays at once).

**Key terms:**
- `shape`: tuple of dimensions (rows, cols, ...)
- `dtype`: data type of the array elements (e.g., `float64`, `int32`)
- `ndim`: number of dimensions (rank)
- `size`: total number of elements
- `itemsize`: bytes per element


In [ ]:

a1 = np.array([1, 2, 3], dtype=np.int64)          # 1D array
a2 = np.array([[1.0, 2.0], [3.0, 4.0]])           # 2D array
a3 = np.zeros((2, 3))                              # zeros
a4 = np.ones((2, 3))                               # ones
a5 = np.full((2, 3), 7)                            # filled with 7
a6 = np.arange(0, 10, 2)                           # 0..8 step 2
a7 = np.linspace(0, 1, 5)                          # 5 points 0..1
a8 = rng.integers(0, 10, size=(2,3))               # random ints

print("a2:", a2)
print("shape:", a2.shape, "ndim:", a2.ndim, "dtype:", a2.dtype)
print("size:", a2.size, "itemsize:", a2.itemsize, "nbytes:", a2.nbytes)



## 2) Dtypes & Casting

NumPy arrays are **homogeneous**: all elements have the same dtype.
You can convert (cast) dtypes explicitly.


In [ ]:

b = np.array([1, 2, 3], dtype=np.int32)
print(b, b.dtype)

c = b.astype(np.float64)
print(c, c.dtype)

d = np.array([1.2, 2.7, 3.1])
print("Rounded:", np.rint(d), "Floored:", np.floor(d), "Ceiled:", np.ceil(d))



## 3) Indexing & Slicing

**Indexing** gets elements; **slicing** gets views of subarrays.

- 1D: `a[i]`, `a[start:stop:step]`
- 2D: `A[row, col]`, `A[r1:r2, c1:c2]`
- Negative indices count from the end.


In [ ]:

v = np.array([10, 11, 12, 13, 14, 15])
print(v[0], v[-1], v[1:4], v[::2])

M = np.arange(1, 13).reshape(3, 4)  # 3x4 matrix 1..12
print("M:", M)
print("Element M[1,2]:", M[1,2])
print("Row slice M[1,:]:", M[1, :])
print("Col slice M[:,2]:", M[:, 2])
print("Submatrix M[0:2, 1:3]:", M[0:2, 1:3])



## 4) Views vs Copies

**Important:** Slices return **views** (no data copied). Modifying a view affects the original array.
Use `.copy()` if you need an independent array.


In [ ]:

A = np.arange(10)
B = A[2:7]      # view
B[0] = 99       # modifies A as well
print("A after modifying B:", A)

C = A[2:7].copy()
C[0] = -1
print("A unchanged by C:", A)
print("C:", C)



## 5) Axis Explained

`axis` tells NumPy **which dimension** to operate along.

- `axis=0`: down the **rows** (operate **column-wise**)
- `axis=1`: across the **columns** (operate **row-wise**)
- Higher dimensions follow the same idea.

**Example:** Summing a 2D array by `axis`.


In [ ]:

X = np.arange(1, 13).reshape(3,4)
print("X:", X)
print("Sum axis=0 (columns):", X.sum(axis=0))  # col sums
print("Sum axis=1 (rows):", X.sum(axis=1))     # row sums
print("Global sum (no axis):", X.sum())



## 6) Broadcasting

**Definition:** Automatic expansion of arrays with **compatible shapes** during operations.
Rules (match from rightmost dim):
1. Dimensions equal → compatible
2. One of them is 1 → stretch (repeat) to match
3. Otherwise → not compatible

**Examples:**


In [ ]:

A = np.array([[1,2,3],[4,5,6]])     # shape (2,3)
b = np.array([10,20,30])            # shape (3,)
print(A + b)                        # b broadcast to (2,3)

col = np.array([[1],[2]])           # shape (2,1)
print(A * col)                      # col broadcast to (2,3)

# Incompatible example (uncomment to see error):
# np.ones((2,3)) + np.ones((4,1))



## 7) Reshaping, Flattening, Transpose

- `reshape(r, c, ...)`: change shape without changing data
- `ravel()` / `flatten()`: 1D view or copy
- `T`: transpose (swap axes for 2D)


In [ ]:

Y = np.arange(12)
print("reshape(3,4):", Y.reshape(3,4))
print("ravel():", Y.reshape(3,4).ravel())
print("transpose:", np.arange(6).reshape(2,3).T)



## 8) Stacking & Splitting

**Stacking:** `np.concatenate`, `np.vstack`, `np.hstack`  
**Splitting:** `np.split`, `np.vsplit`, `np.hsplit`


In [ ]:

a = np.array([1,2,3])
b = np.array([4,5,6])
print("hstack:", np.hstack([a, b]))

A = np.arange(1,7).reshape(2,3)
B = np.arange(7,13).reshape(2,3)
print("vstack:", np.vstack([A,B]))

C = np.arange(10)
left, right = np.split(C, [4])
print("split at index 4:", left, right)



## 9) Boolean & Fancy Indexing

- **Boolean indexing:** pass a boolean mask of same length/shape
- **Fancy indexing:** pass arrays/lists of indices

Useful for filtering rows, conditional assignment, sampling columns.


In [ ]:

data = rng.integers(0, 20, size=10)
mask = data % 2 == 0      # even numbers
print("data:", data)
print("even:", data[mask])

# Conditional assignment
data2 = data.copy()
data2[data2 < 10] = -1
print("conditional assign:", data2)

# Fancy indexing
M = np.arange(1,17).reshape(4,4)
rows = [0, 2]
cols = [1, 3]
print("Fancy pick:", M[rows, cols])



## 10) Aggregations & Arg Functions

Common reductions: `sum`, `mean`, `std`, `var`, `min`, `max`, `median`.  
Index of extrema: `argmin`, `argmax`.  
Sort helpers: `sort`, `argsort`.


In [ ]:

Z = rng.normal(size=(3,5))
print("Z:", Z)
print("mean overall:", Z.mean())
print("mean by column:", Z.mean(axis=0))
print("std by row:", Z.std(axis=1))

flat = Z.ravel()
print("argmax (flat index):", flat.argmax(), "max value:", flat.max())
print("argsort (indices):", np.argsort(flat))



## 11) Sorting & Searching

- `np.sort(a)`: returns a sorted copy
- `a.sort()`: sorts in place
- `np.argsort(a)`: indices that would sort `a`
- `np.searchsorted(sorted_a, value)`: insertion index to keep order


In [ ]:

w = rng.integers(0, 50, size=10)
print("w:", w)
print("sorted copy:", np.sort(w))

w.sort()
print("w after in-place sort:", w)

idx = np.searchsorted(w, 25)
print("Insert 25 at index:", idx, "to keep order")



## 12) Missing Values & Numerical Stability

- Missing values: `np.nan` (not-a-number), infinities: `np.inf`, `-np.inf`
- Use `np.isnan`, `np.isfinite`, `np.nanmean`, `np.nanstd`, etc.
- Numerical stability tips: avoid subtracting similar large numbers; prefer `logaddexp` for log-sum-exp.


In [ ]:

arr = np.array([1.0, np.nan, 2.0, np.inf, -np.inf, 3.0])
print("isnan:", np.isnan(arr))
print("isfinite:", np.isfinite(arr))

x = np.array([1.0, np.nan, 2.0, 3.0])
print("nanmean:", np.nanmean(x), "nanstd:", np.nanstd(x))

# log-sum-exp trick using numpy helpers
a = np.array([1000.0, 1001.0, 1002.0])
# naive exp would overflow; use subtract max and add back in log-space
m = a.max()
stable_logsumexp = m + np.log(np.sum(np.exp(a - m)))
print("stable log-sum-exp:", stable_logsumexp)



## 13) Vectorization & UFuncs

**Vectorization:** Replace slow Python loops with fast array operations.  
**UFuncs:** Universal functions that operate elementwise (e.g., `np.add`, `np.sin`, `np.exp`).

Benefits: speed, clearer code, fewer bugs.


In [ ]:

# Loop vs vectorized
N = 1_00000
x = rng.normal(size=N)

# Python loop (slow)
def py_square(xx):
    out = np.empty_like(xx)
    for i in range(len(xx)):
        out[i] = xx[i] * xx[i]
    return out

# Vectorized
vec = x * x

# Sanity check equality
np.allclose(py_square(x), vec)



### Timing Tip
Use Jupyter's `%timeit` magic (uncomment in a live notebook) to compare performance:
```python
# %timeit py_square(x)
# %timeit x * x
```
Vectorized operations are typically **10x–100x faster**.



## 15) Linear Algebra (Linalg)

Core operations:
- Dot products: `a @ b` or `np.dot(a, b)`
- Matrix multiply: `A @ B`
- Solve linear systems: `np.linalg.solve(A, b)` (preferred over `inv`)
- Norms: `np.linalg.norm(x, ord=2)`
- Eigenvalues/eigenvectors: `np.linalg.eig(A)`
- SVD: `np.linalg.svd(A)`


In [ ]:

A = rng.normal(size=(3,3))
b = rng.normal(size=3)

# Solve A x = b
x = np.linalg.solve(A, b)
residual = A @ x - b

print("solution x:", x)
print("residual (should be ~0):", residual)

# Norms
v = np.array([3.0, 4.0])
print("L2 norm:", np.linalg.norm(v))

# Eigen (not guaranteed real for non-symmetric A)
vals, vecs = np.linalg.eig(A)
print("eigvals:", vals)

# SVD
U, S, VT = np.linalg.svd(A, full_matrices=False)
print("singular values:", S)



## 16) Random Numbers (Generator API)

Prefer `numpy.random.Generator` for new code:
- `integers`, `random`, `normal`, `uniform`, `choice`, `permutation`, etc.
- Set a seed for reproducibility.


In [ ]:

rng = np.random.default_rng(123)

print("integers:", rng.integers(0, 10, size=5))
print("uniform:", rng.uniform(0, 1, size=3))
print("normal:", rng.normal(loc=0, scale=1, size=3))

# Random sampling without replacement
items = np.array(list("ABCDEFGHIJ"))
print("choice:", rng.choice(items, size=4, replace=False))

# Shuffle / permutation
p = rng.permutation(10)  # returns a permuted array of indices 0..9
print("permutation indices:", p)



## 17) `where`, `clip`, `piecewise`

- `np.where(cond, x, y)`: choose elements by condition
- `np.clip(a, min, max)`: limit values to a range
- `np.piecewise(x, condlist, funclist)`: apply piecewise functions


In [ ]:

x = np.linspace(-3, 3, 7)
y = np.where(x >= 0, x**2, -1)   # if x>=0 use x^2 else -1
print("where example:", y)

z = np.array([-10, -1, 0, 2, 100])
print("clipped:", np.clip(z, 0, 10))

pw = np.piecewise(x,
                  [x < 0, x >= 0],
                  [lambda t: -t, lambda t: t**2])
print("piecewise example:", pw)



## 18) Broadcasting: Shape Mechanics (Deep Dive)

Use `np.newaxis` (alias `None`) to add dimensions for broadcasting.


In [ ]:

u = np.array([1,2,3])         # shape (3,)
v = np.array([10,20,30])      # shape (3,)

# Make u a column, v a row to build an outer sum
uc = u[:, np.newaxis]         # shape (3,1)
vr = v[np.newaxis, :]         # shape (1,3)

outer_sum = uc + vr           # shape (3,3)
print("outer sum:", outer_sum)



## 19) Memory Layout & Strides (Overview)

- Arrays are stored in contiguous (C-order) or Fortran-order memory.
- `arr.strides` shows how many bytes to step in each dimension.
- Most users don't need to change this, but it's useful to understand performance.


In [ ]:

Q = np.arange(12).reshape(3,4)
print("Q strides:", Q.strides, "itemsize:", Q.itemsize)

QT = Q.T
print("Q.T strides:", QT.strides)



## 20) Best Practices & Pitfalls

- Prefer **vectorized** operations over Python loops.
- Use `np.linalg.solve` instead of forming `np.linalg.inv(A) @ b`.
- Watch out for **views vs copies** when slicing—use `.copy()` if needed.
- Be mindful of **dtypes** (e.g., integer division vs float).
- For random numbers, use a dedicated `Generator` and pass it around (no globals).
- Avoid chained indexing like `A[A>0][0]` when you mean to modify—use masks directly.



## 21) Mini Exercises (Try Without Scrolling Down 😉)

1. Create an array of shape `(4, 5)` with values 0..19. Extract the last column.
2. Given `arr = np.arange(10)`, set all odd indices to -1 (in-place).
3. Construct a 10×10 multiplication table using broadcasting.
4. Generate 1000 samples from `N(0,1)`, compute mean and std, and compare with theory.
5. Solve `A x = b` for a random 4×4 `A` and random `b`. Verify the residual is ~0.


In [ ]:

# 1)
A = np.arange(20).reshape(4,5)
last_col = A[:, -1]
print("1) last column:", last_col)

# 2)
arr = np.arange(10)
arr[1::2] = -1
print("2) odds set to -1:", arr)

# 3)
row = np.arange(1,11)[:, None]   # (10,1)
col = np.arange(1,11)[None, :]   # (1,10)
table = row * col
print("3) multiplication table (first 5x5):", table[:5,:5])

# 4)
samples = rng.normal(size=1000)
print("4) mean:", samples.mean(), "std:", samples.std())

# 5)
A = rng.normal(size=(4,4))
b = rng.normal(size=4)
x = np.linalg.solve(A, b)
res = A @ x - b
print("5) residual norm:", np.linalg.norm(res))



## 22) Capstone Challenge 🏆

**Goal:** Given a synthetic dataset, standardize features, compute correlation,
fit a least-squares line (1 feature), and report metrics—all in NumPy.

**Steps:**
1. Generate `x` from `Uniform(0, 100)` and target `y = 3.5x + 20 + noise`, where `noise ~ N(0, 50^2)`.
2. Standardize `x` to zero-mean and unit-std: `x_std = (x - mean) / std`.
3. Build design matrix `X = [1, x_std]` and solve `w = (X^T X)^{-1} X^T y`.
4. Compute predictions, MSE, and R².
5. Print `w0` (intercept), `w1` (slope in standardized units), `MSE`, and `R²`.


In [ ]:

# 1) Data
n = 500
x = rng.uniform(0, 100, size=n)
noise = rng.normal(0, 50, size=n)
y = 3.5 * x + 20 + noise

# 2) Standardize x
x_mean = x.mean()
x_std = x.std()
x_s = (x - x_mean) / x_std

# 3) Design matrix and solve (normal equation)
X = np.column_stack([np.ones(n), x_s])
w = np.linalg.inv(X.T @ X) @ (X.T @ y)

# 4) Predictions & metrics
y_hat = X @ w
mse = np.mean((y - y_hat)**2)

ss_res = np.sum((y - y_hat)**2)
ss_tot = np.sum((y - y.mean())**2)
r2 = 1 - ss_res / ss_tot

# 5) Report
print(f"w0 (intercept): {w[0]:.2f}")
print(f"w1 (slope, standardized x): {w[1]:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R^2: {r2:.3f}")
